In [35]:
import pandas as pd
import numpy as np
import warnings
import pickle
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

In [36]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../utils'))
if module_path not in sys.path:
    sys.path.append(module_path)

from helper_functions import *

In [37]:
nb_variables = {'shift': -1}

In [38]:
test = pd.read_csv('Data/modified_test_data.csv', index_col = 0)

In [39]:
test = n_step_shift(test, nb_variables['shift'])

In [40]:
xgb_reg = pickle.load(open('model.pkl', 'rb'))

In [41]:
preds = xgb_reg.predict(test)
preds[preds < 0] = 0
preds = np.round(preds).astype('int32')

In [42]:
preds = fill_future_predictions(preds, nb_variables['shift'])
preds.shape

(260,)

In [43]:
submission = pd.read_csv('../../../dengue_features_test.csv', index_col = 0)
submission = submission.loc['sj']
submission = submission.loc[:, ['year', 'weekofyear']]
submission.head()

,year,weekofyear
city,,
sj,2008,18
sj,2008,19
sj,2008,20
sj,2008,21
sj,2008,22


In [44]:
preds = pd.Series(preds.reshape(-1))
preds.index = submission.index
submission = pd.concat([submission, preds], axis = 1)
submission.columns = ['year', 'weekofyear', 'total_cases']
submission.head()

,year,weekofyear,total_cases
city,,,
sj,2008,18,12
sj,2008,19,11
sj,2008,20,10
sj,2008,21,14
sj,2008,22,10


In [45]:
submission.to_csv('sj_XGBoost_predictions.csv')